In [1]:
import ast
import os

import pandas as pd
from Bio import Entrez

from data_processing import analyze_exons
from fasta_processing import read_single_fasta, dict_align_to_fasta
from parse_psi_blast_results import parse_psiblast_xml, filter_psiblast_hits, update_df, save_files, \
    create_many_cassettes, dict_align_info_analyze

Entrez.email = "artemvaskaa@gmail.com"

## Переопределение функций

In [2]:
def find_cassettes(df):
    ref_exon_len = [37]
    found_protein_ids = {}

    for protein_id in df.protein_id:
        df_exons = analyze_exons(f"{dir}/{protein_id}/exons.fa")
        condition = set(ref_exon_len) & set(df_exons.length.tolist())
        if condition:
            found_protein_ids[protein_id] = condition
    df_found_protein_ids = df[df["protein_id"].isin(found_protein_ids.keys())]  # !!!

    data = {}
    for protein_id in df_found_protein_ids.protein_id:
        df_subset = df_found_protein_ids[df_found_protein_ids["protein_id"] == protein_id]
        org_name = df_subset.org_name.iloc[0]
        df_exons = analyze_exons(f"{dir}/{protein_id}/exons.fa")
        exon_37_idx = df_exons[df_exons["length"] == 37].index[0]
        exon_110_idx = exon_37_idx - 1
        data[f"{org_name}__{protein_id}"] = (df_exons, [exon_110_idx, exon_37_idx])

    return data, df_found_protein_ids

## Введение

In [3]:
# читаем все созданные таблицы
datasets_Cnidaria = pd.read_csv("../results_summary/datasets_Cnidaria_cds_cassette.tsv", sep="\t")
datasets_Ecdysozoa = pd.read_csv("../results_summary/datasets_Ecdysozoa_cds_cassette.tsv", sep="\t")
datasets_Spiralia = pd.read_csv("../results_summary/datasets_Spiralia_cds_cassette.tsv", sep="\t")

# объединяем их в одну
datasets_Protostomia = pd.concat([datasets_Cnidaria, datasets_Ecdysozoa, datasets_Spiralia], ignore_index=True)

# смотрим на value_counts в полученных данных из datasets, чтобы понять, что бластить
datasets_Protostomia.sub_phylum.value_counts()

sub_phylum
Endopterygota      31
Paraneoptera        9
Chelicerata         7
Crustacea           4
Bivalvia            3
Nematoda            3
Anthozoa            2
Gastropoda          2
Scalidophora        1
Polyneoptera        1
Platyhelminthes     1
Name: count, dtype: int64

## Platyhelminthes

[schistosoma_haematobium_platyhelminthes](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_051072204.1%20NXF1_3%20%5Borganism%3DSchistosoma%20haematobium%5D%20%5BGeneID%3D24590952%5D%0AMPHGNHRNKSNHYRWRSHGENDRRIDTSDEYLRRREHGGKFSSGRKSIISGNNVDMLKRAMNMNLIGGSA%0AASVVTAQNSGLAPGEVWVRITIVHGANHPMMDLQQLVTTIVGTQLRFYNTCVEGRNALMHAKIRQKDVQS%0AYRKSLQNLRDPSQGSQLITDITIVPEPRVPSSSDKRNESPNTSPLPETWIEALKQCFVQRYQPTTRSLDL%0ASSLHTDPVLLSQGLYLPLNKQAVVHTLITILKQNQAQLAVLNLSNNRLTHLNAFSPLSSTSAGFIPVSIE%0ARIDLSSNPLSSIPVLSGLRDIVGLVELDLTETPLMSKFNPNDKSFAAKLHTILPTIKRLNGQELPQTVQF%0AAIEQGSDSSKRPPTKPLPQSILGFFPNDEVKIALLSFLKLYLSRYDSKPRGESLLPYYTTVSQLVFSVSP%0AENRFPNSQNVSFTARVEIQNGSDQPTTAYLTTSRLNQAYFLRSRNLLRCRDQSRRRDMVVRGSLAIAHFL%0ADELPTTEHQLESLSVDVAFHSGTQMLFTMGGVFYEVSSMGSSSTNSSSHEKSVRKVLRCFTRTMILIAPG%0AGHIVQDDYIVSNPTTSLCKKYITEMATRCKQDSQASNQQQNVLSSDPSAPEVKENIVIEFSRRTGMNIPF%0ASRQCLEEYEWNANAALTAFETMNLAGKIPPEAFSV&JOB_TITLE=schistosoma_haematobium_platyhelminthes&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Platyhelminthes%20%28taxid%3A6157%29&EQ_MENU1=Schistosoma%20haematobium%20(taxid:6185)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&ADV_VIEW=on&BOOKMARK=on)

In [4]:
phylum = "Platyhelminthes"
dir = f"../Sequences_protein_id/{phylum}"

In [5]:
# # читаем результат BLAST
# name_of_blast_res = f"../Blast_res/psi_blast/schistosoma_haematobium_{phylum.lower()}.xml"
#
# # парсим, фильтруем и добавляем новые столбцы с координатами, именем организма и др.
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
#
# # создаем подпапку в заданной выше директории и сохраняем таблицу, чтобы потом не ждать
# os.makedirs(f"{dir}", exist_ok=True)
# df.to_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index=True)
#
# # закомментить после выполнения!!!

In [6]:
# сохраняем в заданную выше директорию
# save_files(df, dir)

# закомментить после выполнения!!!

In [7]:
# читаем сохраненную ранее таблицу
df = pd.read_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

# получаем словарь для созданию кассет и protein_id для тех организмов, у кого нашелся экзон 37
data, df_found_protein_ids = find_cassettes(df)

# создаем кассеты и заодно получаем интроны для будущего анализа
introns = create_many_cassettes(dir, data)

In [8]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, "cds", dir)
df_cds.equal_to_cds.unique()

Mesocestoides_corti__VDD84209.1: start codon not in the beginning of sequence


array([ True])

In [9]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds
df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(df_found_protein_ids, "cds_cassette", dir)
df_cds_cassette

Mesocestoides_corti__VDD84209.1: start codon not in the beginning of sequence


,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
0,Schistosoma_guineensis__CAH8656942.1,Schistosoma_guineensis,981,False,980,1,652,239
1,Schistosoma_curassoni__CAH8663826.1,Schistosoma_curassoni,981,False,980,1,652,239
2,Schistosoma_bovis__CAH8671530.1,Schistosoma_bovis,981,False,980,1,652,239
3,Schistosoma_margrebowiei__CAH8667553.1,Schistosoma_margrebowiei,981,False,980,1,650,239
4,Schistosoma_intercalatum__CAH8646207.1,Schistosoma_intercalatum,981,False,980,1,652,239
5,Schistosoma_rodhaini__CAH8677847.1,Schistosoma_rodhaini,981,False,980,1,671,239
6,Schistosoma_japonicum__TNN16962.1,Schistosoma_japonicum,978,False,977,1,847,239
7,Schistosoma_turkestanicum__CAH8587689.1,Schistosoma_turkestanicum,978,False,977,1,905,239
8,Paragonimus_westermani__KAF8560478.1,Paragonimus_westermani,993,False,992,1,13971,239
9,Taenia_crassiceps__KAL5103217.1,Taenia_crassiceps,981,False,980,1,278,242


In [10]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
11,Heterobilharzia_americana__CAH8668306.1,Heterobilharzia_americana,984,False,983,1,2163,239
24,Schistosoma_mattheei__CAH8638585.1,Schistosoma_mattheei,981,False,980,1,649,239
26,Schistosoma_mattheei__CAH8638513.1,Schistosoma_mattheei,981,False,980,1,651,239
29,Heterobilharzia_americana__CAH8650739.1,Heterobilharzia_americana,672,False,671,1,2162,132


In [11]:
# удаляем дубликаты

# -----------------
# 29 - Heterobilharzia_americana__CAH8650739.1 - дубликат
# 26 - Schistosoma_mattheei__CAH8638513.1 - дубликат
# -----------------

indices_to_drop = [29, 26]
df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [12]:
# создаем папку и файлы для выравнивания
os.makedirs(f"../Alignment/psi_blast/{phylum}", exist_ok=True)

# закидываем в конец словаря референсную последовательность (вбить вручную)
dict_align_cds_cassette["Schistosoma_haematobium_reference"] = read_single_fasta("../Datasets/Spiralia/schistosoma_haematobium_1/ncbi_dataset/data/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.aln")

# закомментить последнюю строчку после выполнения!!!

In [13]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/psi_blast_{phylum}_cds_cassette.tsv", sep="\t", index=False)

## Mollusca/Bivalvia

[mya_arenaria](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_052767696.1%20LOC128208251%20%5Borganism%3DMya%20arenaria%5D%20%5BGeneID%3D128208251%5D%0AMAEFRVTTDRDGSRSFNDHDDRWTRGGGSGRYRGRGGNFNRRPRRGGGYGYGGSSYNRGRGGYSGGRGRG%0AGDRDGPGPRSRLDDDGDETMGNSETKDIARYNPYGRSSRGNFRGNSKYRSDNRNRPTQIGGDAGIFKRLG%0ALPLDRKTMGDSDWYKVTIPWGKKTEKDFILKSINDHIDVPFVPTYFHYEDKNAVFYVNDSRAAEGLKSTT%0AKRVTMPNGYKMTILVKPSQPPNIPMGKEEIDKLKVCMSNRYDPATKALNLSCLHTDQELAQSNLFMCLAR%0APQVMSNVVKVIKENIPELVQLDVSNNKLQSLEHLGGLVPSTPDMKVLNLANNKINMLEELRKVQKWKIDW%0ALTLDGNPLCDKFNDHTAYVSGVRRLFPKVLKLDTTDLPPPITFDIEARTDLPKSKDSYFPNDTVKNGVVK%0AFLKDYFLVYDSDDRTGLAGAYHETAMFSLSTSYNPTVQNKQTSLSTYIDETRNLLRVYKDTSRKFKTLKN%0AGNKIVAQLCLLPKTQHDPNSFVVDCNFATAQMISFNIQGVFKEVDKKSDKPPMRAFSRTFVTVASGSGMV%0AIVNDVLTVTNASPDQIQTAFKNPAPTPSSSPIPQTSPTEPFAAAGLTEIQTQMVASFMNDSRMNSEWSAK%0ACLVQNNWEYAKAGQNFLELQQKGLIPPEAFKT&JOB_TITLE=mya_arenaria_bivalvia&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Bivalvia%20%28taxid%3A6544%29&EQ_MENU1=Mya%20arenaria%20(taxid:6604)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NEW_VIEW=on&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&ADV_VIEW=on&BOOKMARK=on)

[crassostrea_virginica_bivalvia](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_052767696.1%20LOC128208251%20%5Borganism%3DMya%20arenaria%5D%20%5BGeneID%3D128208251%5D%0AMAEFRVTTDRDGSRSFNDHDDRWTRGGGSGRYRGRGGNFNRRPRRGGGYGYGGSSYNRGRGGYSGGRGRG%0AGDRDGPGPRSRLDDDGDETMGNSETKDIARYNPYGRSSRGNFRGNSKYRSDNRNRPTQIGGDAGIFKRLG%0ALPLDRKTMGDSDWYKVTIPWGKKTEKDFILKSINDHIDVPFVPTYFHYEDKNAVFYVNDSRAAEGLKSTT%0AKRVTMPNGYKMTILVKPSQPPNIPMGKEEIDKLKVCMSNRYDPATKALNLSCLHTDQELAQSNLFMCLAR%0APQVMSNVVKVIKENIPELVQLDVSNNKLQSLEHLGGLVPSTPDMKVLNLANNKINMLEELRKVQKWKIDW%0ALTLDGNPLCDKFNDHTAYVSGVRRLFPKVLKLDTTDLPPPITFDIEARTDLPKSKDSYFPNDTVKNGVVK%0AFLKDYFLVYDSDDRTGLAGAYHETAMFSLSTSYNPTVQNKQTSLSTYIDETRNLLRVYKDTSRKFKTLKN%0AGNKIVAQLCLLPKTQHDPNSFVVDCNFATAQMISFNIQGVFKEVDKKSDKPPMRAFSRTFVTVASGSGMV%0AIVNDVLTVTNASPDQIQTAFKNPAPTPSSSPIPQTSPTEPFAAAGLTEIQTQMVASFMNDSRMNSEWSAK%0ACLVQNNWEYAKAGQNFLELQQKGLIPPEAFKT&JOB_TITLE=mya_arenaria_bivalvia&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Bivalvia%20%28taxid%3A6544%29&EQ_MENU1=Mya%20arenaria%20(taxid:6604)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NEW_VIEW=on&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&ADV_VIEW=on&BOOKMARK=on)

[magallana_gigas_bivalvia](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_052767696.1%20LOC128208251%20%5Borganism%3DMya%20arenaria%5D%20%5BGeneID%3D128208251%5D%0AMAEFRVTTDRDGSRSFNDHDDRWTRGGGSGRYRGRGGNFNRRPRRGGGYGYGGSSYNRGRGGYSGGRGRG%0AGDRDGPGPRSRLDDDGDETMGNSETKDIARYNPYGRSSRGNFRGNSKYRSDNRNRPTQIGGDAGIFKRLG%0ALPLDRKTMGDSDWYKVTIPWGKKTEKDFILKSINDHIDVPFVPTYFHYEDKNAVFYVNDSRAAEGLKSTT%0AKRVTMPNGYKMTILVKPSQPPNIPMGKEEIDKLKVCMSNRYDPATKALNLSCLHTDQELAQSNLFMCLAR%0APQVMSNVVKVIKENIPELVQLDVSNNKLQSLEHLGGLVPSTPDMKVLNLANNKINMLEELRKVQKWKIDW%0ALTLDGNPLCDKFNDHTAYVSGVRRLFPKVLKLDTTDLPPPITFDIEARTDLPKSKDSYFPNDTVKNGVVK%0AFLKDYFLVYDSDDRTGLAGAYHETAMFSLSTSYNPTVQNKQTSLSTYIDETRNLLRVYKDTSRKFKTLKN%0AGNKIVAQLCLLPKTQHDPNSFVVDCNFATAQMISFNIQGVFKEVDKKSDKPPMRAFSRTFVTVASGSGMV%0AIVNDVLTVTNASPDQIQTAFKNPAPTPSSSPIPQTSPTEPFAAAGLTEIQTQMVASFMNDSRMNSEWSAK%0ACLVQNNWEYAKAGQNFLELQQKGLIPPEAFKT&JOB_TITLE=mya_arenaria_bivalvia&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Bivalvia%20%28taxid%3A6544%29&EQ_MENU1=Mya%20arenaria%20(taxid:6604)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NEW_VIEW=on&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&ADV_VIEW=on&BOOKMARK=on)

#### mya_arenaria

In [14]:
phylum = "Bivalvia"
dir = f"../Sequences_protein_id/{phylum}"

In [15]:
# # читаем результат BLAST
# name_of_blast_res = f"../Blast_res/psi_blast/schistosoma_haematobium_{phylum.lower()}.xml"
#
# # парсим, фильтруем и добавляем новые столбцы с координатами, именем организма и др.
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
#
# # создаем подпапку в заданной выше директории и сохраняем таблицу, чтобы потом не ждать
# os.makedirs(f"{dir}", exist_ok=True)
# df.to_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index=True)
#
# # закомментить после выполнения!!!

In [16]:
# сохраняем в заданную выше директорию
# save_files(df, dir)

# закомментить после выполнения!!!

In [17]:
# читаем сохраненную ранее таблицу
df = pd.read_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

# получаем словарь для созданию кассет и protein_id для тех организмов, у кого нашелся экзон 37
data, df_found_protein_ids = find_cassettes(df)

# создаем кассеты и заодно получаем интроны для будущего анализа
introns = create_many_cassettes(dir, data)

In [18]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, "cds", dir)
df_cds.equal_to_cds.unique()

Dreissena_polymorpha__KAH3852511.1: start codon not in the beginning of sequence
Sinanodonta_woodiana__KAL3874362.1: no stop codon found


array([ True])

In [19]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds
df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(df_found_protein_ids, "cds_cassette", dir)
df_cds_cassette

Dreissena_polymorpha__KAH3852511.1: start codon not in the beginning of sequence


,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
0,Mercenaria_mercenaria__XP_053373641.1,Mercenaria_mercenaria,1107,False,1106,1,1690,110
1,Dreissena_polymorpha__XP_052275796.1,Dreissena_polymorpha,1107,False,1106,1,2207,110
2,Ruditapes_philippinarum__XP_060562241.1,Ruditapes_philippinarum,1095,False,1094,1,1646,110
3,Mactra_antiquata__KAL4236273.1,Mactra_antiquata,840,False,839,1,2319,110
4,Mytilus_coruscus__CAC5418521.1,Mytilus_coruscus,1089,False,1088,1,1234,110
5,Potamilus_streckersoni__KAK3583320.1,Potamilus_streckersoni,1071,False,1070,1,4567,110
6,Mytilus_californianus__XP_052058679.1,Mytilus_californianus,1089,False,1088,1,1248,110
7,Saccostrea_echinata__XP_061162409.1,Saccostrea_echinata,1098,False,1097,1,1556,110
8,Mytilus_edulis__CAG2249952.1,Mytilus_edulis,1089,False,1088,1,1360,110
9,Mytilus_edulis__XP_071161937.1,Mytilus_edulis,1089,False,1088,1,1360,110


In [20]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
5,Potamilus_streckersoni__KAK3583320.1,Potamilus_streckersoni,1071,False,1070,1,4567,110
6,Mytilus_californianus__XP_052058679.1,Mytilus_californianus,1089,False,1088,1,1248,110
8,Mytilus_edulis__CAG2249952.1,Mytilus_edulis,1089,False,1088,1,1360,110
9,Mytilus_edulis__XP_071161937.1,Mytilus_edulis,1089,False,1088,1,1360,110
19,Argopecten_irradians__XP_069116938.1,Argopecten_irradians,1170,False,1169,1,5057,110
20,Argopecten_irradians__XP_069116939.1,Argopecten_irradians,1170,False,1169,1,5057,110
21,Magallana_gigas__XP_034325678.2,Magallana_gigas,1035,False,1034,1,1537,110
22,Magallana_gigas__XP_065942772.1,Magallana_gigas,1035,False,1034,1,1537,110
23,Sinanodonta_woodiana__KAL3887057.1,Sinanodonta_woodiana,1083,False,1082,1,6312,101
24,Potamilus_streckersoni__KAK3601259.1,Potamilus_streckersoni,1077,False,1076,1,5690,101


In [21]:
# удаляем дубликаты

# -----------------
# 19 - Argopecten_irradians__XP_069116938.1 - дубликат
# 21 - Magallana_gigas__XP_034325678.2 - дубликат
# 6 - Mytilus_californianus__XP_052058679.1 - дубликат
# 8 - Mytilus_edulis__CAG2249952.1 - дубликат
# 24 - Potamilus_streckersoni__KAK3601259.1 - дубликат
# 23, 25 - Sinanodonta_woodiana__KAL3887057.1, Sinanodonta_woodiana__KAL3887113.1 - трипликат
# -----------------

indices_to_drop = [19, 21, 6, 8, 24, 23, 25]
df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [22]:
# создаем папку и файлы для выравнивания
os.makedirs(f"../Alignment/psi_blast/{phylum}", exist_ok=True)

# закидываем в конец словаря референсную последовательность (вбить вручную)
dict_align_cds_cassette["Mya_arenaria_reference"] = read_single_fasta("../Datasets/Bivalvia/mya_arenaria_0/ncbi_dataset/data/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.aln")

# закомментить последнюю строчку после выполнения!!!

In [23]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/psi_blast_{phylum}_cds_cassette.tsv", sep="\t", index=False)

## Mollusca/Gastropoda

[gigantopelta_aegis](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_041353365.1%20LOC121371499%20%5Borganism%3DGigantopelta%20aegis%5D%20%5BGeneID%3D121371499%5D%0AMSSLTVTAGRDGSRSFGGHDDRWSGKQRNQRRGRGKSYYSRGSYNRGGNKGPRGGGVNPRSRFTDEDDDV%0ANMDSSSGHINRRFDPYGQGGGWNRRNRGRGSFKGGRYHGDFQSHKDTGAMQRMGLPVSRQNQWYKITIPH%0AGKKTGKDEILKLINSVIGAPFQPVYFHYQNKDALFYVKDWDQATALRRTSKMITLPSGFKMIVVVSPCSP%0APVIPMDKESIEKLQKRMSERYDPASKCLDLSSLYQDEVLSASGLYLALNRANTMSNVVKIIQENIPELVG%0ALDLSSNRLLSLSHMTDLVSAAPFVTKLNIGKNQLRSIEELQKIEGWKLLQLVLDGNDLCDRYKERSEYIS%0AMVRKRFPKVINLDGHELPPPITFDLETTTVIPPSKGSYFMNDEVKTIVVKFLKEYYTIYDSDNRQPLFEA%0AYHEQAVFCIATAYNGTLDYKQPSLTDYLQESRNIFRVKDTARREKSIKNGRLPVVSQLCLLPKTTHDPNS%0AFVVDINIVRPTLLSFSLTGIFKETESKSDKPPIRAFNRLFVTVPCGSGMVITNDVLTITNASPEQSQSAF%0AKSTGPTPSSSPVSSAPPAVPSTSLSPIPGTSAVPNKEMIQRFSAESGMNAEWSHKCLEENGWNYKKAALV%0AFTELHSQGKIPQDAFVK&JOB_TITLE=gigantopelta_aegis_gastropoda&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Gastropoda%20%28taxid%3A6448%29&EQ_MENU1=Gigantopelta%20aegis%20(taxid:1735272)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NCBI_GI=false&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&SHOW_CDS_FEATURE=false&ADV_VIEW=on&BOOKMARK=on)

In [24]:
phylum = "Gastropoda"
dir = f"../Sequences_protein_id/{phylum}"

In [25]:
# # читаем результат BLAST
# name_of_blast_res = f"../Blast_res/psi_blast/schistosoma_haematobium_{phylum.lower()}.xml"
#
# # парсим, фильтруем и добавляем новые столбцы с координатами, именем организма и др.
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
#
# # создаем подпапку в заданной выше директории и сохраняем таблицу, чтобы потом не ждать
# os.makedirs(f"{dir}", exist_ok=True)
# df.to_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index=True)
#
# # закомментить после выполнения!!!

In [26]:
# сохраняем в заданную выше директорию
# save_files(df, dir)

# закомментить после выполнения!!!

In [27]:
# читаем сохраненную ранее таблицу
df = pd.read_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

# получаем словарь для созданию кассет и protein_id для тех организмов, у кого нашелся экзон 37
data, df_found_protein_ids = find_cassettes(df)

# создаем кассеты и заодно получаем интроны для будущего анализа
introns = create_many_cassettes(dir, data)

In [28]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, "cds", dir)
df_cds.equal_to_cds.unique()

Physella_acuta__XP_059147995.1: start codon not in the beginning of sequence
Candidula_unifasciata__CAG5136897.1: start codon not in the beginning of sequence


array([ True])

In [29]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds
df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(df_found_protein_ids, "cds_cassette", dir)
df_cds_cassette

Physella_acuta__XP_059147995.1: start codon not in the beginning of sequence
Candidula_unifasciata__CAG5136897.1: start codon not in the beginning of sequence


,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
0,Haliotis_cracherodii__XP_071079091.1,Haliotis_cracherodii,1041,False,1040,1,2506,110
1,Haliotis_rufescens__XP_046327842.1,Haliotis_rufescens,1041,False,1040,1,2505,110
2,Haliotis_asinina__XP_067680995.1,Haliotis_asinina,1038,False,1037,1,2375,110
3,Patella_vulgata__XP_055957145.1,Patella_vulgata,1119,False,1118,1,1384,110
4,Patella_caerulea__KAK6181635.1,Patella_caerulea,1092,False,1091,1,1362,110
5,Patella_caerulea__KAK6181636.1,Patella_caerulea,1092,False,1091,1,1362,110
6,Littorina_saxatilis__XP_070180466.1,Littorina_saxatilis,1095,False,1076,19,6746,218
7,Batillaria_attramentaria__KAK7466614.1,Batillaria_attramentaria,1005,False,1004,1,8614,110
8,Lymnaea_stagnalis__CAL1544423.1,Lymnaea_stagnalis,1074,False,1073,1,2705,221
9,Pomacea_canaliculata__PVD26032.1,Pomacea_canaliculata,471,False,470,1,255,56


In [30]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
4,Patella_caerulea__KAK6181635.1,Patella_caerulea,1092,False,1091,1,1362,110
5,Patella_caerulea__KAK6181636.1,Patella_caerulea,1092,False,1091,1,1362,110
14,Bulinus_truncatus__KAH9514235.1,Bulinus_truncatus,1131,False,1130,1,1873,221
15,Biomphalaria_glabrata__XP_055880571.1,Biomphalaria_glabrata,987,False,986,1,1889,221
16,Bulinus_truncatus__KAH9514233.1,Bulinus_truncatus,1650,False,1649,1,1873,221
18,Biomphalaria_glabrata__XP_055880572.1,Biomphalaria_glabrata,987,False,986,1,1889,221


In [31]:
# удаляем дубликаты

# -----------------
# 15 - Biomphalaria_glabrata__XP_055880571.1 - дубликат
# 16 - Bulinus_truncatus__KAH9514233.1 - дубликат
# 4 - Patella_caerulea__KAK6181635.1 - дубликат
# -----------------

indices_to_drop = [15, 16, 4]
df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [32]:
# создаем папку и файлы для выравнивания
os.makedirs(f"../Alignment/psi_blast/{phylum}", exist_ok=True)

# закидываем в конец словаря референсную последовательность (вбить вручную)
dict_align_cds_cassette["Gigantopelta_aegis_reference"] = read_single_fasta("../Datasets/Gastropoda/gigantopelta_aegis_1/ncbi_dataset/data/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.aln")

# закомментить последнюю строчку после выполнения!!!

In [33]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/psi_blast_{phylum}_cds_cassette.tsv", sep="\t", index=False)

## Ecdysozoa/Nematoda

[caenorhabditis_elegans](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3ENP_001129880.1%20nxf-1%20%5Borganism%3DCaenorhabditis%20elegans%5D%20%5BGeneID%3D191736%5D%0AMNRKGFGGHRDAKQLSRTKNRFARLDPDTQSRYEDDDEPAVPVRASLTSASSRGRGGSSRGFGQSAASIA%0ANTGVRNADIVYKCRATGAAKKVDAKWLIKQLNQIIENFKPLLWTDNARGDFEWYVRDEDTASTIRANNRR%0AVVHKESGTRVEFYTSKVPAPWMKLKREEIEIIHRVVDKRHNAENRVLDLSNFHEDEEFKAKDMMMNLTKG%0ANVMLTVLDHIDDKYGNIVALSLSNNRIRHLDYASALVSIAKFVMELDLSHNHISTEKELEKFAGLPVERF%0AFFEGNPVVESFTQRAAYISYIHQSFPRCNMLDGVEVQPLVVGPDLDIHDAMPFRAGYYPNPQIRVLVEQF%0AVTSYFDFYDGPDGQRTRRNLHNAYDADASTFSLTIEHLRGSSHARHHNDECFAQYAGVSHNVLKQERFAR%0AHRASRSARGAMDIAVALSKLPTSSHMRDTFIVDVFLQSNDLLGFTVQGLFCDGDLTQTPSPSFFSRSFLV%0ASPRENDSVAVISDQLFITVASLDRLEKFKKLYDQSIANGAAVEQVSAVQIAQIGVNGMGFDGAPALPIRE%0AEMIKAMCQFSGMIPPFSEKCLADCAWNFDFACQKFNEIKSSVPAEAFAH&JOB_TITLE=caenorhabditis_elegans_nematoda&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Nematoda%20%28taxid%3A6231%29&EQ_MENU1=Caenorhabditis%20elegans%20(taxid:6239)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NCBI_GI=false&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&SHOW_CDS_FEATURE=false&ADV_VIEW=on&BOOKMARK=on)

In [34]:
phylum = "Nematoda"
dir = f"../Sequences_protein_id/{phylum}"

In [35]:
# # читаем результат BLAST
# name_of_blast_res = f"../Blast_res/psi_blast/schistosoma_haematobium_{phylum.lower()}.xml"
#
# # парсим, фильтруем и добавляем новые столбцы с координатами, именем организма и др.
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
#
# # создаем подпапку в заданной выше директории и сохраняем таблицу, чтобы потом не ждать
# os.makedirs(f"{dir}", exist_ok=True)
# df.to_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index=True)
#
# # закомментить после выполнения!!!

In [36]:
# сохраняем в заданную выше директорию
# save_files(df, dir)

# закомментить после выполнения!!!

In [37]:
# читаем сохраненную ранее таблицу
df = pd.read_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

# получаем словарь для созданию кассет и protein_id для тех организмов, у кого нашелся экзон 37
data, df_found_protein_ids = find_cassettes(df)

# создаем кассеты и заодно получаем интроны для будущего анализа
introns = create_many_cassettes(dir, data)

In [38]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, "cds", dir)
df_cds.equal_to_cds.unique()

Angiostrongylus_cantonensis__KAE9413040.1: no stop codon found
Mesorhabditis_belari__CAJ0960930.1: no stop codon found
Pristionchus_fissidentatus__GMT09523.1: start codon not in the beginning of sequence
Mesorhabditis_spiculigera__CAJ0568410.1: no stop codon found
Pristionchus_pacificus__KAF8386643.1: no stop codon found
Pristionchus_entomophagus__GMS77887.1: start codon not in the beginning of sequence
Pristionchus_pacificus__KAF8382567.1: start codon not in the beginning of sequence
Onchocerca_ochengi__VDK67817.1: no stop codon found
Aphelenchoides_avenae__KAH7707721.1: start codon not in the beginning of sequence


array([ True])

In [39]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds
df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(df_found_protein_ids, "cds_cassette", dir)
df_cds_cassette

Pristionchus_fissidentatus__GMT09523.1: start codon not in the beginning of sequence
Pristionchus_entomophagus__GMS77887.1: start codon not in the beginning of sequence
Pristionchus_pacificus__KAF8382567.1: start codon not in the beginning of sequence
Aphelenchoides_avenae__KAH7707721.1: start codon not in the beginning of sequence


,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
0,Caenorhabditis_nigoni__PIC28742.1,Caenorhabditis_nigoni,1140,False,926,214,142,110
1,Caenorhabditis_briggsae__UMM37086.1,Caenorhabditis_briggsae,1143,False,926,217,145,110
2,Caenorhabditis_brenneri__EGT34035.1,Caenorhabditis_brenneri,1095,False,920,175,130,110
3,Caenorhabditis_brenneri__EGT56062.1,Caenorhabditis_brenneri,1143,False,968,175,130,245
4,Caenorhabditis_sp._36_PRJEB53466__CAI2353387.1,Caenorhabditis_sp._36_PRJEB53466,1134,False,929,205,133,110
5,Caenorhabditis_angaria__CAI5450371.1,Caenorhabditis_angaria,993,False,914,79,96,110
6,Caenorhabditis_bovis__CAB3397759.1,Caenorhabditis_bovis,1236,False,920,316,235,110
7,Caenorhabditis_bovis__CAB3397758.1,Caenorhabditis_bovis,1236,False,920,316,235,110
8,Caenorhabditis_auriculariae__CAD6196466.1,Caenorhabditis_auriculariae,1047,False,902,145,156,110
9,Necator_americanus__ETN84757.1,Necator_americanus,1050,False,914,136,243,110


In [40]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
2,Caenorhabditis_brenneri__EGT34035.1,Caenorhabditis_brenneri,1095,False,920,175,130,110
3,Caenorhabditis_brenneri__EGT56062.1,Caenorhabditis_brenneri,1143,False,968,175,130,245
6,Caenorhabditis_bovis__CAB3397759.1,Caenorhabditis_bovis,1236,False,920,316,235,110
7,Caenorhabditis_bovis__CAB3397758.1,Caenorhabditis_bovis,1236,False,920,316,235,110
10,Nippostrongylus_brasiliensis__WKY10434.1,Nippostrongylus_brasiliensis,1164,False,848,316,235,110
12,Nippostrongylus_brasiliensis__VDL77853.1,Nippostrongylus_brasiliensis,1257,False,935,322,235,110
13,Haemonchus_contortus__XGW01888.1,Haemonchus_contortus,1362,False,1058,304,220,110
14,Haemonchus_contortus__XGW01889.1,Haemonchus_contortus,1362,False,1058,304,220,110
20,Diploscapter_pachys__PAV87020.1,Diploscapter_pachys,948,False,938,10,662,110
22,Nippostrongylus_brasiliensis__WKY10433.1,Nippostrongylus_brasiliensis,1164,False,848,316,235,110


In [41]:
# удаляем дубликаты

# -----------------
# 30 - Acanthocheilonema_viteae__KAL4002522.1 - дубликат
# 46, 45 - Aphelenchoides_avenae__KAH7716608.1, Aphelenchoides_avenae__KAH7703299.1 - трипликат
# 38 - Brugia_pahangi__VDN82903.1 - дубликат
# 7 - Caenorhabditis_bovis__CAB3397758.1 - дубликат
# 3 - Caenorhabditis_brenneri__EGT56062.1 - дубликат
# 23 - Diploscapter_pachys__PAV81848.1 - дубликат
# 34 - Ditylenchus_destructor__KAI1727416.1 - дубликат
# 13 - Haemonchus_contortus__XGW01888.1 - дубликат
# 12, 22 - Nippostrongylus_brasiliensis__VDL77853.1, Nippostrongylus_brasiliensis__WKY10433.1 - трипликат
# 49 - Steinernema_carpocapsae__TKR60559.1 - дубликат
# 39 - Wuchereria_bancrofti__VDM22199.1 - дубликат
# -----------------

indices_to_drop = [30, 46, 45, 38, 7, 3, 23, 34, 13, 12, 22, 49, 39]
df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [42]:
# создаем папку и файлы для выравнивания
os.makedirs(f"../Alignment/psi_blast/{phylum}", exist_ok=True)

# закидываем в конец словаря референсную последовательность (вбить вручную)
dict_align_cds_cassette["Caenorhabditis_elegans_reference"] = read_single_fasta("../Datasets/Nematoda/caenorhabditis_elegans_0/ncbi_dataset/data/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.aln")

# закомментить последнюю строчку после выполнения!!!

In [43]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/psi_blast_{phylum}_cds_cassette.tsv", sep="\t", index=False)

## Ecdysozoa/Scalidophora

[priapulus_caudatus](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_014677623.1%20LOC106817467%20%5Borganism%3DPriapulus%20caudatus%5D%20%5BGeneID%3D106817467%5D%0AMSYQITTNRDGDRNFYGNRGRRRGRGGNGNQSGGGPTPRSRLDAVDEEGDIDMGGGAGRQVHFNRYTPYG%0AGGGGRRPYRGGPRGDRGGYRGNSSYQPPSTMSDRAGSSRNSLERLGLPLQRGGRRGSGNYNKDRHDERQR%0ASVWYKIMIPYGREMGKDFILRSLTTECSVPFTPISFRLDGKMAIFNVEGWDTAEALMKVSRLVTAPSGFK%0AMLVKATPSGPPMPQNLSPTILEQVKEVLNKRYNSELKTLNLTDFHNDPDFKAKKLYLPLNRQNVILSVID%0ALIGKTASEVVALDFTNNKLPNCDSISTLYTKTPKLKGLNLSKNYIRSEYDLDRIKDLNLQELILEGNPLC%0ASQFKDKMSYVSAIRKRFPKVIKLDGLDLPPPIGFDVETHLELPKSKESFFVNDVVKTLVLRFIQEYFMVY%0ADSNDRQQLLNAYHEQAFFSMFAVRKDNSRGPYLQEYLSESRNMIKVKGFERRSKLLKRGRLAVVSMLTTL%0APKTQHDPTSFCIDINHYSAQLLSFTVTGVFKEGDKSSPPVRSFTRVFVVVPVGDGLCIVNEQMCINNATV%0AEQVQASFKSPTPTPSSSPSYPQPGPSGLTSSPVTGAPLVAVNSPSSLTVEQKKVMVEQFMKDSGMNSEWS%0AAKCLEENAWNYEGAGQVFLELRKVGSIPMEAFTKS&JOB_TITLE=priapulus_caudatus_scalidophora&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Scalidophora%20%28taxid%3A1215728%29&EQ_MENU1=Priapulus%20caudatus%20(taxid:37621)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NCBI_GI=false&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&SHOW_CDS_FEATURE=false&ADV_VIEW=on&BOOKMARK=on)

No significant similarity found.

## Cnidaria/Anthozoa

[nematostella_vectensis](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&USER_FORMAT_DEFAULTS=on&SET_SAVED_SEARCH=true&PAGE=Proteins&PROGRAM=blastp&QUERY=%3EXP_001631177.1%20LOC5510737%20%5Borganism%3DNematostella%20vectensis%5D%20%5BGeneID%3D5510737%5D%0AMAGIFGRAMKDVAFSVTTSGDGSRTFNSNIGDGMEDDRQSNRGRGRGGYNNNQGSYRSRGRGRGRGRGRG%0AGNNRGRGGRGGRQDANPRSYLADEEDDESMGDADDNTGGYTSRYTPYGARPPSRRGYQHDDRSGGAGGGI%0AKSRLGQRQHSGWKSFDSDWHRVVVNRAKIHDKEWLIKRLQSSSEEAFQPVEFHYMGESAVFFVEGSRAAE%0AALKRVSHSITVKDGSKLIITVRPSQKPFKSHKGGAGGGGGSSQWSPENEQVLKECLSNRYNPQTKTMDLT%0ADMFHDEVLKANNVFGALNKYPLAQEILKLIGENCPDVESLDVSNNRLFQLDHFKDLATQAPGIKCLNLSN%0ATMLKSIDELDKVKGLKELVTLNLTGNTFCKSFEGKSSAYVSAVRSRFPKVTNLDGNELPPPIGFDLQTST%0AVLPTVQGSYIPDPAVKDLVLKFLEQYFKIYDSGDRQPLLDAYHDQAIFSMCVDTQANSSTKGSRGPSLGP%0AYMRNSRNMKRVTETDHRAALIKHNRLSVVAMLNDFPGTKHDLASFVVDINLTLSTIISFSVRGLFMEEDK%0ATARSFTRVFVAVPAAGGKALSIINDELHIRNASSSQAEKLASAASASIITPNVGSGIMAPAPSATPLPAA%0ATTVPAAGMQNLTPQQQQQMVLQFSNQSQMNPEWSFKCLSENGWNYEKSAEIFTSLQAAGSIPPDAFVKK&JOB_TITLE=nematostella_vectensis_anthozoa&GAPCOSTS=11%201&DATABASE=nr&BLAST_PROGRAMS=psiBlast&MAX_NUM_SEQ=100&SHORT_QUERY_ADJUST=on&EXPECT=0.05&WORD_SIZE=3&MATRIX_NAME=BLOSUM62&COMPOSITION_BASED_STATISTICS=2&I_THRESH=0.005&EQ_MENU=Anthozoa%20%28taxid%3A6101%29&EQ_MENU1=Nematostella%20vectensis%20(taxid:45351)&ORG_EXCLUDE1=on&NUM_ORG=2&PROG_DEFAULTS=on&SHOW_OVERVIEW=on&SHOW_LINKOUT=on&ALIGNMENT_VIEW=Pairwise&MASK_CHAR=2&MASK_COLOR=1&GET_SEQUENCE=on&NCBI_GI=false&NUM_OVERVIEW=100&DESCRIPTIONS=100&ALIGNMENTS=100&FORMAT_OBJECT=Alignment&FORMAT_TYPE=HTML&SHOW_CDS_FEATURE=false&ADV_VIEW=on&BOOKMARK=on)

In [44]:
phylum = "Anthozoa"
dir = f"../Sequences_protein_id/{phylum}"

In [45]:
# # читаем результат BLAST
# name_of_blast_res = f"../Blast_res/psi_blast/schistosoma_haematobium_{phylum.lower()}.xml"
#
# # парсим, фильтруем и добавляем новые столбцы с координатами, именем организма и др.
# df = parse_psiblast_xml(name_of_blast_res)
# df = filter_psiblast_hits(df, min_qc=0, min_ident=0, min_sbjct_len=500)
# df = update_df(df)
#
# # создаем подпапку в заданной выше директории и сохраняем таблицу, чтобы потом не ждать
# os.makedirs(f"{dir}", exist_ok=True)
# df.to_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index=True)
#
# # закомментить после выполнения!!!

In [46]:
# сохраняем в заданную выше директорию
# save_files(df, dir)

# закомментить после выполнения!!!

In [47]:
# читаем сохраненную ранее таблицу
df = pd.read_csv(f"{dir}/df_{phylum.lower()}.tsv", sep="\t", index_col=0,
                 converters={11: ast.literal_eval, 13: ast.literal_eval})

# получаем словарь для созданию кассет и protein_id для тех организмов, у кого нашелся экзон 37
data, df_found_protein_ids = find_cassettes(df)

# создаем кассеты и заодно получаем интроны для будущего анализа
introns = create_many_cassettes(dir, data)

In [48]:
# проверяем, чтобы в столбце equal_to_cds было везде True - это значит, что cds начинается со старт-кодона и заканчивается стоп-кодоном
df_cds, dict_align_cds = dict_align_info_analyze(df_found_protein_ids, "cds", dir)
df_cds.equal_to_cds.unique()

Exaiptasia_diaphana__XP_020894941.1: start codon not in the beginning of sequence
Porites_lobata__CAH3150435.1: start codon not in the beginning of sequence


array([ True])

In [49]:
# смотрим на позиции стоп-кодона в интроне в его сохранении в cds
df_cds_cassette, dict_align_cds_cassette = dict_align_info_analyze(df_found_protein_ids, "cds_cassette", dir)
df_cds_cassette

Exaiptasia_diaphana__XP_020894941.1: start codon not in the beginning of sequence
Porites_lobata__CAH3150435.1: start codon not in the beginning of sequence


,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
0,Actinia_tenebrosa__XP_031555922.1,Actinia_tenebrosa,1152,False,1142,10,173,116
1,Pocillopora_verrucosa__XP_058954564.2,Pocillopora_verrucosa,1152,False,1118,34,390,116
2,Porites_lutea__XP_073247758.1,Porites_lutea,1173,False,1112,61,711,116
3,Pocillopora_damicornis__XP_027046453.1,Pocillopora_damicornis,1164,False,1118,46,392,116
4,Porites_lutea__XP_073247755.1,Porites_lutea,1173,False,1112,61,711,116
5,Porites_lutea__XP_073247757.1,Porites_lutea,1173,False,1112,61,711,116
6,Pocillopora_damicornis__XP_027046452.1,Pocillopora_damicornis,1164,False,1118,46,392,116
7,Montipora_foliosa__XP_068703612.1,Montipora_foliosa,1155,False,1124,31,907,116
8,Montipora_foliosa__XP_068703613.1,Montipora_foliosa,1155,False,1124,31,907,116
9,Pocillopora_verrucosa__XP_058954563.2,Pocillopora_verrucosa,1152,False,1118,34,390,116


In [50]:
# ищем дубликаты
df_cds_cassette[df_cds_cassette["org_name"].duplicated(keep=False)]

,org_name_protein_id,org_name,stop_codon_pos,equal_to_cds,cassette_intron_start,intron_length_to_stop_codon,intron_length,first_exon_length
1,Pocillopora_verrucosa__XP_058954564.2,Pocillopora_verrucosa,1152,False,1118,34,390,116
2,Porites_lutea__XP_073247758.1,Porites_lutea,1173,False,1112,61,711,116
3,Pocillopora_damicornis__XP_027046453.1,Pocillopora_damicornis,1164,False,1118,46,392,116
4,Porites_lutea__XP_073247755.1,Porites_lutea,1173,False,1112,61,711,116
5,Porites_lutea__XP_073247757.1,Porites_lutea,1173,False,1112,61,711,116
6,Pocillopora_damicornis__XP_027046452.1,Pocillopora_damicornis,1164,False,1118,46,392,116
7,Montipora_foliosa__XP_068703612.1,Montipora_foliosa,1155,False,1124,31,907,116
8,Montipora_foliosa__XP_068703613.1,Montipora_foliosa,1155,False,1124,31,907,116
9,Pocillopora_verrucosa__XP_058954563.2,Pocillopora_verrucosa,1152,False,1118,34,390,116
11,Acropora_millepora__XP_029179300.2,Acropora_millepora,1167,False,1127,40,682,116


In [51]:
# удаляем дубликаты

# -----------------
# 12 - Acropora_digitifera__XP_015766187.1
# 11 - Acropora_millepora__XP_029179300.2
# 7 - Montipora_foliosa__XP_068703612.1
# 6 - Pocillopora_damicornis__XP_027046452.1
# 9 - Pocillopora_verrucosa__XP_058954563.2
# 2, 4, 5 - Porites_lutea__XP_073247758.1, Porites_lutea__XP_073247755.1, Porites_lutea__XP_073247757.1
# -----------------

indices_to_drop = [12, 11, 7, 6, 9, 2, 4, 5]
df_cds_cassette = df_cds_cassette.drop(indices_to_drop)

In [52]:
# создаем папку и файлы для выравнивания
os.makedirs(f"../Alignment/psi_blast/{phylum}", exist_ok=True)

# закидываем в конец словаря референсную последовательность (вбить вручную)
dict_align_cds_cassette["Nematostella_vectensis_reference"] = read_single_fasta("../Datasets/Anthozoa/nematostella_vectensis_2/ncbi_dataset/data/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.fa")
dict_align_to_fasta(dict_align_cds_cassette, f"../Alignment/psi_blast/{phylum}/cds_cassette.aln")

# закомментить последнюю строчку после выполнения!!!

In [53]:
# сохраняем табличку для последующей загрузки в results_summary ноутбук
df_cds_cassette.to_csv(f"../results_summary/psi_blast_{phylum}_cds_cassette.tsv", sep="\t", index=False)